# Save and Restore
[save and restore](https://www.tensorflow.org/guide/saved_model)

## Save and restore variables
TensorFlow Variables are the best way to represent shared, persistent state manipulated by your program. 
- The ```tf.train.Saver``` constructor adds save and restore ops to the graph for all, or a specified list, of the variables in the graph. 
- The ```Saver``` object provides methods to run these ops, specifying paths for the checkpoint files to write to or read from. ```Saver``` restores all variables already defined in your model.

TensorFlow saves variables in binary checkpoint files that map variable names to tensor values.

### Save variables
Create a ```Saver``` with ```tf.train.Saver()``` to manage all variables in the model.

In [ ]:
import tensorflow as tf

# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

In [ ]:
# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  # Do some work with the model.
  inc_v1.op.run()
  dec_v2.op.run()
  # Save the variables to disk.
  save_path = saver.save(sess, "./model/model.ckpt")
  print("Model saved in path: %s" % save_path)

### Restore variables
The ```tf.train.Saver``` object not only saves variables to checkpoint files, it also restores variables. 

In [ ]:
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "./model/model.ckpt")
  print("Model restored.")
  # Check the values of the variables
  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())


#### Choose variables to save and restore
It is sometimes useful to explicitly specify names for variables in the checkpoint files. It is also sometimes useful to only save or restore a subset of the variables used by a model. 

You can specify the names and variables to save or load by passing to the ```tf.train.Saver()``` constructor either of the following:
- A list of variables (which will be stored under their own names).
- A Python dictionary in which keys are the names to use and the values are the variables to manage.

In [ ]:
tf.reset_default_graph()
# Create some variables.
v1 = tf.get_variable("v1", [3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", [5], initializer = tf.zeros_initializer)

# Add ops to save and restore only `v2` using the name "v2"
saver = tf.train.Saver({"v2": v2})

# Use the saver object normally after that.
with tf.Session() as sess:
  # Initialize v1 since the saver will not.
  v1.initializer.run()
  saver.restore(sess, "./model/model.ckpt")

  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())

Notes:
- Can crearte as many ```Saver``` objecdts as possible to save and restore different subsets of the model variables.
- If only restore a subset of the model variables at the start of a session, have to run an initialize op for the other variables.
- Can use the ```inspect_checkpoint``` library to inspect the variables in a checkpoint.
- By default, Saver uses the value of the ```tf.Variable.name``` property for each variable. However, ```Saver``` can optionally choose names for the variables in the checkpoint files.

## Build and load a SavedModel
### Simple save
The easiest way to create a ```SavedModel``` is to use the ```tf.saved_model.simple_save``` function:
```python
simple_save(session,
            export_dir,
            inputs={"x": x, "y": y},
            outputs={"z": z})
```
### Manually build a SavedModel

### Loading a SavedModel in Python
The ```load``` operation requires:
- The session in which to restore the graph definition and variables.
- The tags used to identify the MetaGraphDef to load.
- The location (directory) of the SavedModel.

In [ ]:
import tensorflow as tf

# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  # Do some work with the model.
  inc_v1.op.run()
  dec_v2.op.run()
  # Save the variables to disk.
  save_path = saver.save(sess, "./model/model.ckpt")
  print("Model saved in path: %s" % save_path)

In [2]:
import tensorflow as tf
# saver = tf.train.Saver()
# tf.reset_default_graph()
# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
g_1 = tf.Graph()
with g_1.as_default():    
  v3 = tf.get_variable("v3", shape=[5], initializer = tf.zeros_initializer)
  # Add an op to initialize the variables.
  init_op = tf.global_variables_initializer()
  # Restore variables from disk.
  with tf.Session() as sess:
    sess.run(init_op)
    saver = tf.train.import_meta_graph('./model/model.ckpt.meta')
    saver.restore(sess, "./model/model.ckpt")
    print("Model restored.")
    print(g_1.get_operations())

INFO:tensorflow:Restoring parameters from ./model/model.ckpt
Model restored.
[<tf.Operation 'v3/Initializer/zeros' type=Const>, <tf.Operation 'v3' type=VariableV2>, <tf.Operation 'v3/Assign' type=Assign>, <tf.Operation 'v3/read' type=Identity>, <tf.Operation 'init' type=NoOp>, <tf.Operation 'v1/Initializer/zeros' type=Const>, <tf.Operation 'v1' type=VariableV2>, <tf.Operation 'v1/Assign' type=Assign>, <tf.Operation 'v1/read' type=Identity>, <tf.Operation 'v2/Initializer/zeros' type=Const>, <tf.Operation 'v2' type=VariableV2>, <tf.Operation 'v2/Assign' type=Assign>, <tf.Operation 'v2/read' type=Identity>, <tf.Operation 'add/y' type=Const>, <tf.Operation 'add' type=Add>, <tf.Operation 'Assign' type=Assign>, <tf.Operation 'sub/y' type=Const>, <tf.Operation 'sub' type=Sub>, <tf.Operation 'Assign_1' type=Assign>, <tf.Operation 'init_1' type=NoOp>, <tf.Operation 'save/filename/input' type=Const>, <tf.Operation 'save/filename' type=PlaceholderWithDefault>, <tf.Operation 'save/Const' type=Plac